# Clasificador de peliculas


El presente labook aborda el desarrollo de un sistema de clasificación multietiqueta de géneros cinematográficos a partir de un conjunto de datos compuesto por las variables year, plot y genres. El flujo de trabajo incluye la preparación, limpieza y normalización del texto, así como la vectorización mediante técnicas avanzadas de NLP. Posteriormente, se integran modelos preentrenados del ecosistema Hugging Face, basados en arquitecturas transformer de última generación, combinados con redes neuronales para la capa de clasificación final. El objetivo es entrenar y evaluar un modelo capaz de asignar múltiples géneros a cada sinopsis, maximizando métricas de desempeño como el F1-score dentro de un entorno de aprendizaje supervisado.

## Carga de librerias

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
import tensorflow as tf
import numpy as np
import ast

## Carga del dataset

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


## Aplicacion de tecnicas NLP

In [13]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
lemmatize = WordNetLemmatizer()

In [10]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Instanciar herramientas
tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función de limpieza y lematización
def lemmatize_and_clean(text):
    tokens = tokenizer.tokenize(text.lower())
    return [lemmatizer.lemmatize(token, pos='v') for token in tokens if token.isalpha() and token not in stop_words]


In [11]:
print(stop_words)

{'as', "he'd", 'what', "hasn't", "he'll", "haven't", 'against', 'needn', "it'll", 'themselves', 'ours', 've', 'from', 'ourselves', 'by', 'not', 'only', 'hasn', "i'll", 'myself', "it'd", "it's", 'doesn', 's', 'you', "hadn't", 'than', 'those', "you'd", 'yourselves', "we've", 'now', 'before', 'once', 'shan', 'but', "won't", 'ain', 'yourself', 'doing', "aren't", "needn't", 'so', 'same', 't', 'when', 'how', 'himself', "they'd", 'your', 'out', 'its', 'am', 'haven', "you're", "they'll", "we'll", 'should', 'on', "didn't", 'couldn', 'or', 'our', 'has', "mustn't", 'have', 'yours', 'through', 'just', 'can', 'was', "isn't", 'won', 'into', "that'll", 'below', 'this', 'weren', 'him', 'had', 'with', 'shouldn', 'wasn', 'if', 'are', 'again', 'll', 'she', 'i', 'don', 'why', 'a', 'mightn', 'an', 'and', 'most', 'does', 'having', 'herself', 'more', 'own', 'such', 'while', 'over', "don't", 'any', 'each', 'few', 'the', 'who', "i've", 'until', 'isn', "she's", 'under', "they've", 'is', 'because', 'my', "we're"

In [14]:
vect = CountVectorizer(analyzer=lemmatize_and_clean)
X = vect.fit_transform(dataTraining['plot'])
X.shape

(7895, 32411)

Se generaron 7.895 instancias (una por cada sinopsis en dataTraining['plot']).

El vocabulario final contiene 32.411 términos únicos, que pasan a constituir las dimensiones del espacio vectorial.

Cada sinopsis se representa como un vector disperso de longitud 32.411, donde cada componente indica la frecuencia del término correspondiente.

In [15]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
le = MultiLabelBinarizer()
y = le.fit_transform(dataTraining['genres'])

In [16]:
# Dividir entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import AUC

# Convertir a matriz densa para Keras
X_train = X.toarray()
y_train = y
# 0. Limpieza de sesión
K.clear_session()

# Parámetros
vocab_size = 10000
max_len = 200
embedding_dim = 128
num_classes = y_train.shape[1]
learning_rate = 0.001
optimizer     = Adam(learning_rate=learning_rate)
epochs        = 10
batch_size    = 32
l2_lambda     = 1e-4

# Modelo
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.01))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=[AUC(name='auc')]
)

model.build(input_shape=(None, max_len))
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │         1,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,335,128 (5.09 MB)

 Trainable params: 1,335,128 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Entrenamiento con EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score

# Entrenamiento con early stopping
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ],
    verbose=1
)

# Evaluación final: predicciones probabilísticas
# Convert X_test to a dense array to avoid the SparseToDense error
y_pred_proba = model.predict(X_test.toarray())

# Cálculo de ROC AUC (macro y micro)
roc_auc_macro = roc_auc_score(y_test, y_pred_proba, average='macro')
roc_auc_micro = roc_auc_score(y_test, y_pred_proba, average='micro')

print(f'ROC AUC (macro): {roc_auc_macro:.4f}')
print(f'ROC AUC (micro): {roc_auc_micro:.4f}')

Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 183s 923ms/step - auc: 0.7879 - loss: 0.2952 - val_auc: 0.7875 - val_loss: 0.2955
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 178s 899ms/step - auc: 0.7898 - loss: 0.2933 - val_auc: 0.7873 - val_loss: 0.2953
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 178s 898ms/step - auc: 0.7899 - loss: 0.2944 - val_auc: 0.7879 - val_loss: 0.2948
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 178s 898ms/step - auc: 0.7888 - loss: 0.2957 - val_auc: 0.7890 - val_loss: 0.2947
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 177s 897ms/step - auc: 0.7910 - loss: 0.2939 - val_auc: 0.7887 - val_loss: 0.2951
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 177s 896ms/step - auc: 0.7887 - loss: 0.2945 - val_auc: 0.7888 - val_loss: 0.2945
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - auc: 0.7921 - loss: 0.2928 - val_auc: 0.7877 - val_loss: 0.2948
Epoch 8/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 182s 918ms/step - auc: 0.7906 - loss: 0.2959 - val_auc: 0.7880 - val_loss: 0.2946
Epoch 9/10
198/198 ━━━━━━━━

El uso de modelos preentrenados resulta ventajoso porque aprovechan representaciones lingüísticas aprendidas sobre grandes corpus, lo que permite obtener un rendimiento significativamente superior al de los enfoques tradicionales entrenados desde cero, especialmente en tareas con recursos de datos limitados por esa razon se procedera a hacer uso del ecosistema Hugging Face.

## REDES NEURONALES + TRANSFORMER

In [ ]:
import pandas as pd
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss


In [22]:


# ---------------------
# 1. Limpieza y lematización para otro modelo
# ---------------------
tokenizer_nltk = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def lemmatize_and_clean(text):
    tokens = tokenizer_nltk.tokenize(text.lower())
    return " ".join([lemmatizer.lemmatize(token, pos='v') for token in tokens if token.isalpha() and token not in stop_words])

# Opción: Si quieres limpiar texto antes de BERT (no obligatorio, pero opcional)
dataTraining['plot_clean'] = dataTraining['plot'].apply(lemmatize_and_clean)



In [ ]:
# ---------------------
# 2. Codificación de etiquetas (géneros)
# ---------------------
import ast  # alternativa más segura que eval()
import numpy as np # Import numpy

dataTraining['genres'] = dataTraining['genres'].map(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

mlb = MultiLabelBinarizer()
# Convert to float32 here
dataTraining['label'] = mlb.fit_transform(dataTraining['genres']).astype(np.float32).tolist()
dataTraining=dataTraining.drop_duplicates(subset='plot').reset_index(drop=True)

# ---------------------
# 3. División train/test
# ---------------------
X_train, X_test, y_train, y_test = train_test_split(
    dataTraining['plot'],  # o 'plot' si no quieres limpiar antes
    dataTraining['label'],
    test_size=0.2,
    random_state=42
)


In [ ]:
train_dataset = Dataset.from_dict({'text': list(X_train), 'labels': list(y_train)})
test_dataset  = Dataset.from_dict({'text': list(X_test),  'labels': list(y_test)})

In [ ]:
# ---------------------
# 4. Tokenización BERT
# ---------------------
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return bert_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test  = test_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/6315 [00:00<?, ? examples/s]

Map:   0%|          | 0/1579 [00:00<?, ? examples/s]

In [ ]:
# ---------------------
# 5. Modelo BERT multietiqueta
# ---------------------
num_labels = len(mlb.classes_)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels, problem_type="multi_label_classification")




Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ---------------------
# 6. Métrica personalizada
# ---------------------
def compute_metrics(pred):
    logits, labels = pred
    # En multilabel, necesitas aplicar sigmoid
    probs = 1 / (1 + np.exp(-logits))

    try:
        auc = roc_auc_score(labels, probs, average="macro")
    except ValueError:
        auc = float('nan')  # En caso de que falle por datos vacíos

    loss = log_loss(labels, probs)

    return {
        'roc_auc_macro': auc,
        'log_loss': loss
    }


In [ ]:
# ---------------------
# 7. Configuración del entrenamiento
# ---------------------
training_args = TrainingArguments(
    output_dir="./bert_genre_model_optuna_v2",
    learning_rate=3.1449904984156505e-05,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.02345047974845148,
    warmup_steps=564,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc_macro"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)



In [ ]:
  # ---------------------
  # 8. Entrenamiento
  # ---------------------
  trainer.train()

  # ---------------------
  # 9. Evaluación final
  # ---------------------
  trainer.evaluate()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Roc Auc Macro,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,0.231500,0.224052,0.847354,3.124958,6.343500,248.915000,31.213000
2,0.195200,0.190065,0.888339,2.719760,6.336300,249.199000,31.249000
3,0.137000,0.181561,0.903923,2.505830,6.353600,248.521000,31.164000
4,0.106100,0.184287,0.903740,2.930735,6.342000,248.976000,31.221000
5,0.085500,0.195550,0.906501,2.723488,6.330000,249.445000,31.279000
6,0.075900,0.204314,0.904230,2.941021,6.343600,248.913000,31.213000
7,0.048000,0.213102,0.904092,3.050194,6.334900,249.254000,31.255000
8,0.029300,0.221337,0.907905,3.090428,6.344100,248.893000,31.210000
9,0.030000,0.227105,0.907110,3.263512,6.357100,248.384000,31.146000
10,0.017500,0.243107,0.905769,3.335074,6.361600,248.206000,31.124000


{'eval_loss': 0.22133655846118927,
 'eval_roc_auc_macro': 0.9079048683106317,
 'eval_log_loss': 3.09042823599602,
 'eval_runtime': 6.3974,
 'eval_samples_per_second': 246.82,
 'eval_steps_per_second': 30.95,
 'epoch': 20.0}

In [ ]:
predictions = trainer.predict(tokenized_test)
y_pred_proba = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()


In [ ]:
# Paso 1: Asegúrate de que no haya valores nulos en la columna 'plot'
dataTesting['plot'] = dataTesting['plot'].fillna('')

# Paso 2: Tokenizar el texto de test
tokenized_dataTesting = bert_tokenizer(
    list(dataTesting['plot']),
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)


In [ ]:
import torch

# 1. Detectar el dispositivo (GPU o CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Enviar el modelo al dispositivo
model.to(device)

# 3. Asegurarse de que los tensores estén también en el mismo dispositivo
tokenized_dataTesting = tokenized_dataTesting.to(device)  # si es un Dataset con tensores


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
# Convert tokenized_dataTesting dictionary to a TensorDataset
# Ensure all tensors are on the correct device
test_input_ids = tokenized_dataTesting['input_ids'].to(device)
test_attention_mask = tokenized_dataTesting['attention_mask'].to(device)
# Add other potential keys like 'token_type_ids' if they exist and are needed by the model
# For simplicity, we'll assume only input_ids and attention_mask are necessary based on the traceback context
test_dataset_tensor = TensorDataset(test_input_ids, test_attention_mask)

# Define a batch size for prediction (can be larger than training batch size if only inference is done)
# Start with a moderate size and reduce if OOM still occurs
prediction_batch_size = 64 # Or 64, 128 depending on available GPU memory

# Create a DataLoader
test_dataloader = DataLoader(test_dataset_tensor, batch_size=prediction_batch_size)

# List to store predictions
all_predictions = []

# Perform predictions in batches
with torch.no_grad():
    for batch in test_dataloader:
        # Unpack the batch and move to the device
        batch_input_ids, batch_attention_mask = batch

        # Make prediction for the current batch
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)

        # Apply sigmoid and append to the list
        batch_probs = torch.sigmoid(outputs.logits).cpu().numpy()
        all_predictions.append(batch_probs)

# Concatenate predictions from all batches
y_pred_proba = np.concatenate(all_predictions, axis=0)



In [ ]:
# Paso 4: Crear DataFrame de resultados con las columnas adecuadas
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Paso 5: Recortar dataTesting para que coincidan los índices si es necesario
# This might not be strictly necessary if y_pred_proba has the same length as dataTesting,
# but keeping it for safety.
dataTesting_aligned = dataTesting.iloc[:len(y_pred_proba)]

# Paso 6: Crear DataFrame con probabilidades
res = pd.DataFrame(y_pred_proba, index=dataTesting_aligned.index, columns=cols)
res.index.name = 'ID'

# Paso 7 (opcional): Ver resultado
print(res.head())

    p_Action  p_Adventure  p_Animation  p_Biography  p_Comedy   p_Crime  \
ID                                                                        
1   0.003666     0.007259     0.000678     0.001306  0.000589  0.007125   
4   0.096450     0.004693     0.003053     0.938300  0.068513  0.973406   
5   0.006944     0.001181     0.002746     0.034599  0.000663  0.972333   
6   0.451207     0.069753     0.000248     0.000624  0.001610  0.011990   
7   0.001709     0.002856     0.002071     0.016185  0.014372  0.002712   

    p_Documentary   p_Drama  p_Family  p_Fantasy  ...  p_Musical  p_Mystery  \
ID                                                ...                         
1        0.000448  0.953342  0.001331   0.038539  ...   0.008203   0.314237   
4        0.014501  0.995690  0.006145   0.002335  ...   0.019394   0.001893   
5        0.011166  0.996515  0.002098   0.003489  ...   0.002344   0.473030   
6        0.000138  0.965599  0.000165   0.003346  ...   0.000870   0.006078   


In [ ]:
res.to_csv("pred_genres_text_RDS.csv", index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
ID,,,,,,,,,,,,,,,,,,,,,
1,0.003666,0.007259,0.000678,0.001306,0.000589,0.007125,0.000448,0.953342,0.001331,0.038539,...,0.008203,0.314237,0.000795,0.976204,0.005423,0.001258,0.000438,0.899192,0.007881,0.003512
4,0.096450,0.004693,0.003053,0.938300,0.068513,0.973406,0.014501,0.995690,0.006145,0.002335,...,0.019394,0.001893,0.007060,0.040046,0.004602,0.009837,0.024653,0.040639,0.004927,0.007282
5,0.006944,0.001181,0.002746,0.034599,0.000663,0.972333,0.011166,0.996515,0.002098,0.003489,...,0.002344,0.473030,0.002996,0.002923,0.002967,0.006004,0.001584,0.986482,0.004587,0.003598
6,0.451207,0.069753,0.000248,0.000624,0.001610,0.011990,0.000138,0.965599,0.000165,0.003346,...,0.000870,0.006078,0.000161,0.100906,0.001582,0.000484,0.000162,0.977483,0.043121,0.002333
7,0.001709,0.002856,0.002071,0.016185,0.014372,0.002712,0.005404,0.081729,0.002560,0.010231,...,0.008549,0.040971,0.002951,0.106505,0.852708,0.014283,0.006552,0.085953,0.002748,0.005100


In [ ]:
from google.colab import files
files.download("pred_genres_text_RDS.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Refinacion
calibracion hiperparametros

In [ ]:
from transformers import BertForSequenceClassification

def model_init():
    return BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=len(mlb.classes_),       # número de géneros
        problem_type="multi_label_classification"
    )


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import IntervalStrategy
import optuna

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 6),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 1000),
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert_genre_optuna",
    eval_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc_macro",
    greater_is_better=True,
)


In [ ]:
trainer = Trainer(
    model=model,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    tokenizer=bert_tokenizer
)
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    n_trials=10,            # Número de pruebas
    study_name="hp_optuna",
    storage=None,           # O bien una URI de sqlite para persistir la búsqueda
    load_if_exists=True
)

print("Mejores hiperparámetros encontrados:", best_run.hyperparameters)


[I 2025-05-26 17:19:10,899] A new study created in memory with name: hp_optuna
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jhon971014 (jhon971014-universidad-de-los-andes) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

[I 2025-05-26 18:00:17,950] Trial 8 pruned. 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/log_loss,▁
eval/loss,▁
eval/roc_auc_macro,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▂▂▁▁▁▁▁▁▂▂▂▁▂▁▃▃▂▅▂▃▃▂▂▂▂▂▂▃▂▃▅▅▃▃▂▅▃▁▃
train/learning_rate,▁▇███████████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
train/loss,█▇▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Epoch,Training Loss,Validation Loss,Roc Auc Macro,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,0.280400,0.280329,0.700256,3.980766,6.181800,255.426000,32.029000
2,0.229300,0.230455,0.840178,3.400738,6.240600,253.022000,31.728000
3,0.203400,0.211604,0.873057,2.941467,6.240600,253.021000,31.728000
4,0.178000,0.198758,0.881549,2.873212,6.229400,253.475000,31.785000
5,0.168200,0.193990,0.888929,2.766629,6.238600,253.101000,31.738000
6,0.161200,0.192532,0.888837,2.743866,6.263100,252.112000,31.614000


[I 2025-05-26 18:07:12,657] Trial 9 finished with value: 3.6327026218329994 and parameters: {'learning_rate': 2.745429372377183e-05, 'num_train_epochs': 6, 'per_device_train_batch_size': 32, 'weight_decay': 0.17875462925006133, 'warmup_steps': 90}. Best is trial 9 with value: 3.6327026218329994.


Mejores hiperparámetros encontrados: {'learning_rate': 2.745429372377183e-05, 'num_train_epochs': 6, 'per_device_train_batch_size': 32, 'weight_decay': 0.17875462925006133, 'warmup_steps': 90}


### regresion logistica

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect= TfidfVectorizer(analyzer=lemmatize_and_clean, max_features=10000, ngram_range=(1, 2))
X = vect.fit_transform(dataTraining['plot'])
X.shape

(7895, 10000)

In [ ]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
le = MultiLabelBinarizer()
y = le.fit_transform(dataTraining['genres'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000))

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred_prob = model.predict_proba(X_test)
y_pred_bin = (y_pred_prob > 0.5).astype(int)  # Umbral ajustable

print(classification_report(y_test, y_pred_bin, target_names=le.classes_))
print("ROC AUC macro:", roc_auc_score(y_test, y_pred_prob, average="macro"))
print("ROC AUC micro:", roc_auc_score(y_test, y_pred_prob, average="micro"))


              precision    recall  f1-score   support

      Action       0.76      0.11      0.19       423
   Adventure       0.96      0.07      0.13       340
   Animation       0.00      0.00      0.00        99
   Biography       0.00      0.00      0.00       130
      Comedy       0.76      0.47      0.58      1028
       Crime       0.88      0.21      0.34       468
 Documentary       0.95      0.16      0.27       129
       Drama       0.68      0.72      0.70      1283
      Family       0.00      0.00      0.00       252
     Fantasy       1.00      0.02      0.04       243
   Film-Noir       0.00      0.00      0.00        57
     History       0.00      0.00      0.00        80
      Horror       0.87      0.09      0.16       300
       Music       1.00      0.04      0.08       123
     Musical       0.00      0.00      0.00        97
     Mystery       1.00      0.02      0.04       242
        News       0.00      0.00      0.00         3
     Romance       0.81    